In [2]:
!pip freeze | grep scikit-learn

scikit-learn==1.3.2


In [3]:
!python -V

Python 3.10.13


In [4]:
import pickle
import pandas as pd

In [5]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/maedr3/.pyenv/versions/3.10.13/envs/3.10/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/maedr3/.pyenv/versions/3.10.13/envs/3.10/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [12]:
year = 2023
month = 3

In [13]:
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [14]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [15]:
import numpy as np

In [16]:
np.std(y_pred)

6.247488852238703

In [18]:
prediction = pd.DataFrame(y_pred)

In [19]:
prediction['ride_id'] = f'{year:04d}/{month:02d}_' + prediction.index.astype('str')

In [21]:
output_file = f"{year:04d}_{month:02d}_predictions.parquet"

In [22]:
prediction.to_parquet(output_file, engine='pyarrow',
                     compression=None, index=False)